In [ ]:
#| default_exp base_model

In [ ]:
#| exporti 
from typing import *
from pydantic import BaseModel as PydanticBaseModel
from pydantic.generics import GenericModel as PydanticGenericModel
import json
from json2html import json2html
from IPython.display import HTML,JSON
import inspect
import yaml
from archetypon.delegates import delegates
import logging
from pandas import DataFrame as PandasDataFrame


# Base Model
> Extending Pydantic's BaseModel with Jupyter-specific utilities

In [ ]:
#|export 

class DataFrame(PandasDataFrame):
    """Subclassed from Pandas DataFrame. Includes classmethods used in Pydantic Validation"""
    
    @classmethod
    def __get_validators__(cls):
        yield cls.validate_dataframe
    
    @classmethod
    def __modify_schema__(cls, field_schema: Dict[str, Any]) -> None:
        pass
    
    @classmethod
    def validate_dataframe(cls,v):

        return PandasDataFrame(v)

In [ ]:

#|exporti

def dict_to_yaml(data: dict) -> str:
    # convert the dictionary to a yaml string
    yaml_str = yaml.dump(data,sort_keys = False)

    return yaml_str

In [ ]:

#|exporti

def pydantic_to_dbt(model: Type[PydanticBaseModel]) -> dict:
    # convert the model to a dictionary
    model_dict = model.schema()

    # create a dictionary for the dbt model
    dbt_model = {
        "version": 2,
        "name": model.__name__.lower(),  # use the model's class name as the table name
        "description":model_dict.get('description'),
        "columns": []
    }
    dbt_model = {k:v for k,v in dbt_model.items() if v or k=='columns'}

    # add the columns from the pydantic model to the dbt model
    for field_name, field in model_dict["properties"].items():

        column = {
            "name": field_name,
            "description":field.get('description'),
            "type": field["type"],
        }
        dbt_model["columns"].append({k:v for k,v in column.items() if v})
    return dbt_model


In [ ]:
#|exporti

class Base():

    @delegates(PydanticBaseModel.json)
    def display_json(self,**kwargs):
        """Helper function to display json in jupyter lab using kwargs passed to pydantic's .json() method"""
        return JSON(
            json.loads(self.json(**kwargs))
        )
    
    @delegates(PydanticBaseModel.json)
    def display_html(self,**kwargs):
        return HTML(
            json2html.convert(self.json(**kwargs))
        )
    
    @classmethod
    @delegates(PydanticBaseModel.schema_json)
    def display_schema_json(cls,**kwargs):
        """Helper function to display schema json in jupyter lab using kwargs passed to pydantic's .json() method"""
        return JSON(
            json.loads(cls.schema_json(**kwargs))
        )

    @classmethod
    @delegates(PydanticBaseModel.schema_json)
    def schema_html(cls,**kwargs):
        return HTML(
            json2html.convert(cls.schema_json(**kwargs))
        )

    @classmethod
    @delegates(PydanticBaseModel.schema)
    def schema_yml(cls,**kwargs):
        dbt = pydantic_to_dbt(cls)

        return dict_to_yaml(dbt)

    def _repr_html_(self):
        try:
            return self.display_html(**self.Display.html).data
        except Exception as e:
            logging.warning(e)
            pass

    def _repr_json_(self):
        try:
            return self.display_json(**self.Display.json).data
        except Exception as e:
            logging.warning(e)
            pass

    class Display:
        json = {}
        html = {}
    
    class Config:
        json_encoders = {
            PandasDataFrame: lambda df: json.loads(df.to_json(date_format='iso'))
        }

In [ ]:
#|export

class BaseModel(PydanticBaseModel,Base):
    """
    Custom implementation of Pydantic's Base Model.

    Includes `_repr_json_` and `_repr_html_` methods for nice displays in Jupyter Lab and Jupyter Notebook, respectively.

    """    
    class Config(Base.Config):
        pass

In [ ]:
import datetime as dt
from pydantic import validator
from dateutil.relativedelta import relativedelta
from pydantic import ValidationError

In [ ]:
class Person(BaseModel):
    name: str
    dob: dt.date
    age: Optional[int] 
    
    @validator('age',always=True)
    def _validate_age(cls,v,values):
        difference_in_years = relativedelta(
            dt.date.today(), 
            values['dob']
        ).years
        if v and v!=difference_in_years:
            raise ValueError("You're lying about your age!")
        return difference_in_years

In [ ]:
me = Person(
    name = 'Humble Chuck',
    dob = '1994-06-11'
)
me

name,Humble Chuck
dob,1994-06-11
age,28


In [ ]:
try:
    me = Person(
        name = 'Humble Chuck',
        dob = '1994-06-11',
        age = 27
    )
except ValidationError as e:
    print(e)

1 validation error for Person
age
  You're lying about your age! (type=value_error)


In [ ]:
#|export 

class GenericModel(PydanticGenericModel,Base):
    """
    Custom implementation of Pydantic's Generic Model.

    Includes `_repr_json_` and `_repr_html_` methods for nice displays in Jupyter Lab and Jupyter Notebook, respectively.

    """    

    class Config(Base.Config):
        pass

In [ ]:
class ModelWithDataFrame(BaseModel):
    df: DataFrame
    

In [ ]:
data = {
    'a':[1],
    'b':[2]
}
dataframe = DataFrame(data) 
ModelWithDataFrame(df=dataframe)

ModelWithDataFrame(df=   a  b
0  1  2)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()